In [72]:
import numpy as np
import pandas as pd
from collections import defaultdict
from backroom import *
for cv_split in range(1, 6):
    train_df = pd.read_csv(f"../../data/interim/train_df_2_fold_{cv_split}.csv")
    val_df = pd.read_csv(f"../../data/interim/val_df_2_fold_{cv_split}.csv")
    test_df = pd.read_csv(f"../../data/interim/test_df_2.csv")
    datasets = [train_df, val_df, test_df]
    def get_risk_score(median_for_all_locations, median, mean, count):
        if count < 30:
            result = mean*count/30 + median_for_all_locations*(30-count)/30
        else:
            result = median
        return result


    # Dual location risk score
    median_for_all_locations = train_df.groupby(['PULocationID', 'DOLocationID']).duration_seconds.mean().quantile(0.5)
    location_mean_lookup = defaultdict(lambda: median, train_df.groupby(['PULocationID', 'DOLocationID']).duration_seconds.mean().to_dict())
    location_median_lookup = defaultdict(lambda: median, train_df.groupby(['PULocationID', 'DOLocationID']).duration_seconds.median().to_dict())
    location_count_lookup = defaultdict(lambda: 0, train_df.groupby(['PULocationID', 'DOLocationID']).duration_seconds.count().to_dict())
    def get_dual_location_risk_score(PULocationID, DOLocationID):
        return get_risk_score(median_for_all_locations, location_median_lookup[(PULocationID, DOLocationID)], location_mean_lookup[(PULocationID, DOLocationID)], location_count_lookup[(PULocationID, DOLocationID)])
    for dataset in datasets:
        dataset['dual_location_risk_score'] = dataset[['PULocationID', 'DOLocationID']].apply(lambda x: get_dual_location_risk_score(*x), axis='columns')

    def get_single_risk_score(column_name):
        # Get a simple risk score
        median_for_all_locations = train_df.groupby([column_name]).duration_seconds.mean().quantile(0.5)
        location_mean_lookup = defaultdict(lambda: median, train_df.groupby([column_name]).duration_seconds.mean().to_dict())
        location_median_lookup = defaultdict(lambda: median, train_df.groupby([column_name]).duration_seconds.median().to_dict())
        location_count_lookup = defaultdict(lambda: 0, train_df.groupby([column_name]).duration_seconds.count().to_dict())
        def get_simple_risk_score(value):
            return get_risk_score(median_for_all_locations, location_median_lookup[(value)], location_mean_lookup[(value)], location_count_lookup[(value)])
        for dataset in datasets:
            dataset[f'{column_name}_risk_score'] = dataset[column_name].apply(lambda x: get_simple_risk_score(str(x)))
    for column in ['PULocationID', 'DOLocationID', 'pickup_day_of_week', 'VendorID', 'passenger_count', 'payment_type_1', 'hour', 'closest_hour', 'closest_hour_and_a_half', 'dayofweek_plus_hour', 'dayofweek_plus_closest_hour', 'dayofweek_plus_closest_hour_and_a_half']:
        get_single_risk_score(column)
    train_df.to_csv(f"../../data/interim/train_df_3_fold_{cv_split}.csv", index=False)
    val_df.to_csv(f"../../data/interim/val_df_3_fold_{cv_split}.csv", index=False)
    test_df.to_csv(f"../../data/interim/test_df_3_fold_{cv_split}.csv", index=False)
    